In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import log_loss
import numpy as np
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict, RandomizedSearchCV

from gc import collect

In [11]:
TARGET = ['identity_hate', 'insult', 'obscene','severe_toxic', 'threat', 'toxic']

In [15]:
data =  pd.read_csv('data/train.csv', sep=',', index_col='id')
data['train'] = 1
data = pd.concat([data, pd.read_csv('data/test.csv', sep=',', index_col='id')], axis=0)
data.train.fillna(0, inplace=True)
print(data.shape)
data.head()

(312735, 8)


,comment_text,identity_hate,insult,obscene,severe_toxic,threat,toxic,train
id,,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0.0,0.0,0.0,0.0,0.0,0.0,1.0
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0.0,0.0,0.0,0.0,0.0,0.0,1.0
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [56]:
pipe = Pipeline([('tfidf', TfidfVectorizer()), ('lg', LogisticRegression())])

In [59]:
space = {
 'lg__C': [.0001, .001, .01, .1, 1.0, 10, 100, 1000],
 'lg__class_weight': [{1:2, 0:1},{1:3, 0:1}, {1:4, 0:1}, 'balanced', None],
 'lg__fit_intercept': [True,False],
 'lg__max_iter': [100, 200, 300, 500, 1000],
 'lg__penalty': ['l2', 'l1'],
 'lg__random_state': [234],
 'lg__tol': [0.0001, .001, .01],
 'lg__warm_start': [False,True],
 #'tfidf__analyzer': 'word',
 'tfidf__binary': [False,True],
 'tfidf__decode_error': ['ignore'],
 #'tfidf__input': 'content',
 'tfidf__lowercase': [False,True],
 'tfidf__max_df': list(np.linspace(0.8, 1.0, 20)),
 'tfidf__max_features': [10000, None],
 'tfidf__min_df': [1, 5, 10, 20, 30, 50, 100],
 'tfidf__ngram_range': [(1, 1), (2, 1), (3, 1), (2, 2), (3, 2),(3, 3), (4, 1)],
 'tfidf__norm': ['l2','l1', None],
 'tfidf__smooth_idf': [False,True],
 #'tfidf__stop_words': None,
 #'tfidf__strip_accents': None,
 'tfidf__sublinear_tf': [False,True],
 #'tfidf__token_pattern': '(?u)\\b\\w\\w+\\b',
 'tfidf__use_idf': [False,True]
 #'tfidf__vocabulary': None
        }

In [ ]:
score = []
rnd = RandomizedSearchCV(pipe, param_distributions=space, n_iter=200, scoring='neg_log_loss', 
                       cv = 5, verbose=3, random_state=234)
for i, tar in enumerate(TARGET):
    print(tar)
    score.append(rnd.fit(data.loc[data.train==1, 'comment_text'], data.loc[data.train==1, tar].values))
    pd.DataFrame(score[i].cv_results_).to_csv('metrics/grid_search_'+tar+'_.csv', sep=';', index=False)

identity_hate
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV] tfidf__use_idf=False, tfidf__sublinear_tf=False, tfidf__smooth_idf=False, tfidf__norm=None, tfidf__ngram_range=(3, 3), tfidf__min_df=10, tfidf__max_features=10000, tfidf__max_df=0.894736842105, tfidf__lowercase=True, tfidf__decode_error=ignore, tfidf__binary=True, lg__warm_start=False, lg__tol=0.01, lg__random_state=234, lg__penalty=l1, lg__max_iter=500, lg__fit_intercept=False, lg__class_weight={1: 2, 0: 1}, lg__C=0.001 
[CV]  tfidf__use_idf=False, tfidf__sublinear_tf=False, tfidf__smooth_idf=False, tfidf__norm=None, tfidf__ngram_range=(3, 3), tfidf__min_df=10, tfidf__max_features=10000, tfidf__max_df=0.894736842105, tfidf__lowercase=True, tfidf__decode_error=ignore, tfidf__binary=True, lg__warm_start=False, lg__tol=0.01, lg__random_state=234, lg__penalty=l1, lg__max_iter=500, lg__fit_intercept=False, lg__class_weight={1: 2, 0: 1}, lg__C=0.001, score=-0.676797, total= 3.1min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.3min remaining:    0.0s


[CV] tfidf__use_idf=False, tfidf__sublinear_tf=False, tfidf__smooth_idf=False, tfidf__norm=None, tfidf__ngram_range=(3, 3), tfidf__min_df=10, tfidf__max_features=10000, tfidf__max_df=0.894736842105, tfidf__lowercase=True, tfidf__decode_error=ignore, tfidf__binary=True, lg__warm_start=False, lg__tol=0.01, lg__random_state=234, lg__penalty=l1, lg__max_iter=500, lg__fit_intercept=False, lg__class_weight={1: 2, 0: 1}, lg__C=0.001 
[CV]  tfidf__use_idf=False, tfidf__sublinear_tf=False, tfidf__smooth_idf=False, tfidf__norm=None, tfidf__ngram_range=(3, 3), tfidf__min_df=10, tfidf__max_features=10000, tfidf__max_df=0.894736842105, tfidf__lowercase=True, tfidf__decode_error=ignore, tfidf__binary=True, lg__warm_start=False, lg__tol=0.01, lg__random_state=234, lg__penalty=l1, lg__max_iter=500, lg__fit_intercept=False, lg__class_weight={1: 2, 0: 1}, lg__C=0.001, score=-0.676957, total= 2.8min


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  8.2min remaining:    0.0s


[CV] tfidf__use_idf=False, tfidf__sublinear_tf=False, tfidf__smooth_idf=False, tfidf__norm=None, tfidf__ngram_range=(3, 3), tfidf__min_df=10, tfidf__max_features=10000, tfidf__max_df=0.894736842105, tfidf__lowercase=True, tfidf__decode_error=ignore, tfidf__binary=True, lg__warm_start=False, lg__tol=0.01, lg__random_state=234, lg__penalty=l1, lg__max_iter=500, lg__fit_intercept=False, lg__class_weight={1: 2, 0: 1}, lg__C=0.001 
[CV]  tfidf__use_idf=False, tfidf__sublinear_tf=False, tfidf__smooth_idf=False, tfidf__norm=None, tfidf__ngram_range=(3, 3), tfidf__min_df=10, tfidf__max_features=10000, tfidf__max_df=0.894736842105, tfidf__lowercase=True, tfidf__decode_error=ignore, tfidf__binary=True, lg__warm_start=False, lg__tol=0.01, lg__random_state=234, lg__penalty=l1, lg__max_iter=500, lg__fit_intercept=False, lg__class_weight={1: 2, 0: 1}, lg__C=0.001, score=-0.676665, total= 2.5min
[CV] tfidf__use_idf=False, tfidf__sublinear_tf=False, tfidf__smooth_idf=False, tfidf__norm=None, tfidf__ng

In [47]:
pd.DataFrame(score[i].cv_results_).to_csv('grid_search_'+tar+'_.csv', sep=';', index=False)